In [ ]:
import netCDF4
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pathlib import Path
# Set the font size and style
plt.rcParams.update({'font.size': 11, 'font.family': 'serif'})


In [ ]:
df_list = []
fp = '../../SnowEx-Data'
lidar_dir = Path(fp)
fps = list(lidar_dir.glob('*.nc'))
fps

for fp in fps:
    # open dataset
    ds = xr.open_dataset(fp)
    #get lidar flight time
    ft = pd.to_datetime(ds.attrs['lidar-flight-time'])
    # 3 days buffer
    dt = pd.Timedelta('3 days')
    # get lidar-sd and lidar-vh
    lidar_sd = ds['lidar-sd'].values.flatten()
    lidar_vh = ds['lidar-vh'].values.flatten()
    # get mean spicy snow depth for the time period
    snow_depth = ds['snow_depth'].sel(time=slice(
        ft - dt, ft + dt)).mean(dim='time').values.flatten()
    #put in dataframe
    df = pd.DataFrame({'lidar_vh': lidar_vh,
                       'lidar_sd': lidar_sd, 'snow_depth': snow_depth})
    #append to df_list
    df_list.append(df)


In [ ]:
merged_df = pd.concat(df_list, ignore_index=True)
merged_df.dropna(inplace= True)

In [ ]:
# Select non-negative records for lidar_sd and lidar_vh variables
merged_df = merged_df[(
    merged_df['lidar_sd'] >= 0) & (merged_df['lidar_vh'] >= 0)]


In [ ]:
merged_df.describe()

In [ ]:
# Define the bin edges and labels for vegetation height classes
bin_edges = [0, 0.2, 2, float('inf')]
class_labels = ['sparse vegetation', 'medium vegetation', 'dense vegetation']

# Bin the vegetation height into classes
merged_df['vegetation_class'] = pd.cut(
    merged_df['lidar_vh'], bins=bin_edges, labels=class_labels)

# Print the updated DataFrame
print(merged_df)


In [ ]:
# Select records where vegetation_class is 'sparse'
merged_df[merged_df['vegetation_class'] == 'sparse vegetation']


In [ ]:
class_colors = {'sparse vegetation': 'red', 'medium vegetation': 'green', 'dense vegetation': 'blue'}

In [ ]:
# Create a list to store the data for each vegetation class
class_data = []

# Iterate over vegetation classes and extract the data
for veg_class, color in class_colors.items():
    class_data.append(merged_df.loc[merged_df['vegetation_class'] == veg_class, [
                      'lidar_sd', 'snow_depth']])

# Set up the figure and axes
fig, axes = plt.subplots(nrows=1, ncols=len(
    class_data), figsize=(12, 6), sharey=True)

# Create violin plots for each vegetation class
for i, (data, color) in enumerate(zip(class_data, class_colors.values())):
    ax = axes[i]
    sns.violinplot(data=data, palette=[color], ax=ax)
    ax.set_xlabel('Variable')
    ax.set_ylabel('Value')
    ax.set_title(list(class_colors.keys())[i])

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:


# Plot scatter plot
plt.figure(figsize=(8, 6))
for veg_class, color in class_colors.items():
    class_data = merged_df[merged_df['vegetation_class'] == veg_class]
    plt.scatter(class_data['lidar_sd'],
                class_data['snow_depth'], c=color, label=veg_class)

# Add labels and legend
plt.xlabel('Lidar SD (m)')
plt.ylabel('Snow Depth (m)')
plt.legend()

# Show the plot
plt.show()


In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Iterate over vegetation classes and plot scatter plots
for i, (veg_class, color) in enumerate(class_colors.items()):
    class_data = merged_df[merged_df['vegetation_class'] == veg_class]
    ax = axes[i]
    ax.scatter(class_data['lidar_sd'], class_data['snow_depth'], c=color)
    ax.set_xlabel('Lidar SD (m)')
    ax.set_ylabel('Spicy Snow Depth (m)')
    ax.set_title(veg_class)

    # Calculate and display the correlation (r-value)
    r_value = np.corrcoef(
        class_data['lidar_sd'], class_data['snow_depth'])[0, 1]
    ax.text(0.05, 0.9, f'r = {r_value:.2f}', transform=ax.transAxes)

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# dataset = xr.open_dataset(
#     '/home/naheemadebisi/geofiles1/home/NaheemAdebisi/PhD/dissertation/spicy-snow/SnowEx-Data/Banner_2020-02-18.sub.nc')
# lidar_vh_values = dataset['lidar-vh'].values.flatten()
# lidar_sd_values = dataset['lidar-sd'].values.flatten()
# snow_depth_values = dataset['snow_depth'].sel(
#     time='2020-02-17T13:49:32').values.flatten()

# # Create a DataFrame to hold the values
# banner_2020_02_18_df = pd.DataFrame({'lidar_vh': lidar_vh_values,
#                   'lidar_sd': lidar_sd_values, 'snow_depth': snow_depth_values})
# banner_2020_02_18_df
